tmp

In [5]:
import pandas as pd
import scipy.io.wavfile as wav
import numpy as np
from os import path

In [6]:
folder = 'wygibasy'

In [75]:
def take_track_names(folder):
    dir_to_song = path.join(folder, "song.txt")
    song = pd.read_csv(dir_to_song, header = None, dtype = str)
    # all unique tracks used in the song
    track_numbers = list(set(song.loc[:, 0]))
    num_of_tracks = len(track_numbers)
    name_of_track = [None] * num_of_tracks
    for i in range(num_of_tracks):
        name_of_track[i] = 'track' + track_numbers[i] + '.txt'
    return name_of_track

In [76]:
def create_all_track(name_of_track, bpm, folder):
    # unique tracks
    num_of_tracks = len(name_of_track)
    dict_of_tracks = {}
    for i in range(num_of_tracks):
        dict_of_tracks[name_of_track[i]] = create_track(name_of_track[i], bpm, folder)
    return dict_of_tracks

In [109]:
name_of_track = take_track_names(folder)
dict_of_tracks = create_all_track(name_of_track, bpm, folder) #ok!

/usr/lib/python3/dist-packages/scipy/io/wavfile.py:172: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [110]:
dict_of_tracks

{'track01.txt': [array([[  6,   4],
         [ -1, -10],
         [  5,  14],
         ..., 
         [  0,   0],
         [  0,   0],
         [  0,   0]], dtype=int16), 235200, 379829],
 'track02.txt': [array([[ 6,  3],
         [ 5,  4],
         [-4, -4],
         ..., 
         [ 0,  0],
         [ 0,  0],
         [ 0,  0]], dtype=int16), 235200, 336035],
 'track03.txt': [array([[ 9,  5],
         [ 2, -7],
         [-2,  6],
         ..., 
         [ 0,  0],
         [ 0,  0],
         [ 0,  0]], dtype=int16), 235200, 379829]}

In [124]:
def create_song(dict_of_tracks, folder):
    dir_to_song = path.join(folder, "song.txt")
    song = pd.read_csv(dir_to_song, header = None, dtype = str)
    # all tracks used in the song in the order
    track_names = 'track' + song.loc[:, 0] + '.txt'
    # the length of the song = sum of raw length of every track + eventually additional ending of the last track
    length_of_song = 0
    for i, j in enumerate(track_names):
        if i != len(track_names) - 1:
            length_of_song += dict_of_tracks[j][1]
        else:
            length_of_song += dict_of_tracks[j][2]
    song = np.zeros((length_of_song, 2))
    for i, j in enumerate(track_names):
        if i == 0:
            where_to_start = 0
            raw_length_of_track = dict_of_tracks[j][1]
            true_length_of_track = dict_of_tracks[j][2]
            song[where_to_start:true_length_of_track, :] += dict_of_tracks[j][0]
            where_to_start += raw_length_of_track 
        else:
            raw_length_of_track_new = dict_of_tracks[j][1]
            true_length_of_track_new = dict_of_tracks[j][2]
            song[where_to_start:(where_to_start + true_length_of_track_new), :] += dict_of_tracks[j][0]
            where_to_start += raw_length_of_track_new
    song = np.array(song, dtype = 'int16')
    return song

In [125]:
piosenka = create_song(dict_of_tracks, folder)

In [126]:
piosenka

array([[  6,   4],
       [ -1, -10],
       [  5,  14],
       ..., 
       [  0,   0],
       [  0,   0],
       [  0,   0]], dtype=int16)

In [128]:
piosenka[2000000:, ]

array([[0, 0],
       [0, 0],
       [0, 0],
       ..., 
       [0, 0],
       [0, 0],
       [0, 0]], dtype=int16)

In [95]:
piosenka.shape #shape ok

(2959679, 2)

In [123]:
max(piosenka[:, 1]) #na pograniczu maksa

32767

In [97]:
bpm = 180
folder = 'wygibasy'
track1 = 'track01.txt'

In [129]:
wav.write('wygibasy/piosenka.wav', 44100, piosenka)

In [130]:
!!aplay 'wygibasy/piosenka.wav'

["Playing WAVE 'wygibasy/piosenka.wav' : Signed 16 bit Little Endian, Rate 44100 Hz, Stereo"]

In [14]:
def concatenate_one_type_of_sample(column_of_track, length_of_column, folder, next_sample):
    column = np.zeros((length_of_column, 2))
    # check the number of sample
    num_of_sample = list(set(column_of_track).difference(['--']))[0]
    sample = wav.read(path.join(folder, 'sample' + num_of_sample + '.wav'))
    # number of bits in the sample
    n = sample[1].shape[0]
    # if length of sample is bigger than number of sounds (?) per bit we add more columns at the end of array
    if n > next_sample:
        how_many_to_add = n - next_sample
        column = np.zeros((length_of_column + how_many_to_add, 2))
    # concatenate sample and 'empty sounds' in order
    for i, j in enumerate(column_of_track):
        if j != '--':
            column[i * next_sample : i * next_sample + n, :] += sample[1]
    column = np.array(column, dtype = 'int16')
    return column

In [15]:
def concatenate_all_samples_in_track(track, bpm, folder):
    nrow_track = track.shape[0]
    ncol_track = track.shape[1]
    # bit per second
    bps = bpm / 60
    # all samples have the same sample rate
    f = 44100
    next_sample = int(f // bps)
    N = next_sample * nrow_track
    list_of_samples = list()
    for k in range(ncol_track):
        column_of_track = track.loc[:, k]
        column = concatenate_one_type_of_sample(column_of_track, N, folder, next_sample)
        list_of_samples.append(column)
    return [N, list_of_samples]

In [16]:
def merge_all_concatenated_samples(list_of_samples):
    k = len(list_of_samples)
    # look for maximum number of rows in all samples
    num_of_rows = [None] * k
    for i in range(k):
        num_of_rows[i] = list_of_samples[i].shape[0]
    max_of_rows = max(num_of_rows)                 
    # append all arrays with zeros to have the same shape
    for i in range(k):             
        n = list_of_samples[i].shape[0]
        if n != max_of_rows:
            list_of_samples[i] = np.vstack((list_of_samples[i], np.zeros((max_of_rows - n, 2))))  
    #mix all sample into one array (average)
    sample_all = np.zeros((max_of_rows, 2))
    for i in range(k):
        sample_all += 1/k * list_of_samples[i]
    sample_all = np.array(sample_all, dtype = 'int16')
    return sample_all

In [33]:
def create_track(track_name, bmp, folder):
    path_to_track = path.join(folder, track_name)
    track = pd.read_csv(path_to_track, sep = ' ', header = None, dtype = str)
    raw_num_of_rows_and_list_of_samples = concatenate_all_samples_in_track(track, bpm, folder)
    raw_num_of_rows = raw_num_of_rows_and_list_of_samples[0]
    track_array = merge_all_concatenated_samples(raw_num_of_rows_and_list_of_samples[1])
    true_num_of_rows = track_array.shape[0]
    return [track_array, raw_num_of_rows, true_num_of_rows]

In [99]:
a = create_track(track1, bpm, folder)
b = create_track('track02.txt', bpm, folder)
c = create_track('track03.txt', bpm, folder)

/usr/lib/python3/dist-packages/scipy/io/wavfile.py:172: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [111]:
a

[array([[  6,   4],
        [ -1, -10],
        [  5,  14],
        ..., 
        [  0,   0],
        [  0,   0],
        [  0,   0]], dtype=int16), 235200, 379829]

In [112]:
b

[array([[ 6,  3],
        [ 5,  4],
        [-4, -4],
        ..., 
        [ 0,  0],
        [ 0,  0],
        [ 0,  0]], dtype=int16), 235200, 336035]

In [113]:
c

[array([[ 9,  5],
        [ 2, -7],
        [-2,  6],
        ..., 
        [ 0,  0],
        [ 0,  0],
        [ 0,  0]], dtype=int16), 235200, 379829]

In [102]:
wav.write('wygibasy/a.wav', 44100, a[0])
wav.write('wygibasy/b.wav', 44100, b[0])
wav.write('wygibasy/c.wav', 44100, c[0])

In [106]:
!!aplay 'wygibasy/a.wav'

["Playing WAVE 'wygibasy/a.wav' : Signed 16 bit Little Endian, Rate 44100 Hz, Stereo"]

In [107]:
!!aplay 'wygibasy/b.wav'

["Playing WAVE 'wygibasy/b.wav' : Signed 16 bit Little Endian, Rate 44100 Hz, Stereo"]

In [108]:
!!aplay 'wygibasy/c.wav'

["Playing WAVE 'wygibasy/c.wav' : Signed 16 bit Little Endian, Rate 44100 Hz, Stereo"]